Import The Libraries📚

In [ ]:
import os
import pandas as pd
import torch
import json

In [ ]:
import os
import ast
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf
import librosa
import librosa.display
import IPython.display as display

In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [ ]:
# model1 = tf.keras.models.load_model("../input/l-a-save-model/modelv0-birdclef.h5")
modelv1 = tf.keras.models.load_model("../input/modelv1/modelv1-birdclef.h5")

In [ ]:
os.getcwd()

SEE THE DATA 🐔:

In [ ]:
path = '/kaggle/input/birdclef-2022/'
os.listdir(path)

train_meta = pd.read_csv(path+'train_metadata.csv')
test_data = pd.read_csv(path+'test.csv')
ebird_data = pd.read_csv(path+'eBird_Taxonomy_v2021.csv')
samp_subm = pd.read_csv(path+'sample_submission.csv')

with open(path+'scored_birds.json') as f:
    scored_birds = json.load(f)

In [ ]:
class_list = train_meta.iloc[:,0]
converter = LabelEncoder()
y = converter.fit_transform(class_list)

**TEST MODEL**

In [ ]:
import csv

2. Features extraction
In the preprocessing of the data, feature extraction is necessary before running the training. The purpose is to define the inputs and outputs of the neural network.

OUTPUT (y): last column which is the label.
You cannot use text directly for training. You will encode these labels with the LabelEncoder() function of sklearn.preprocessing.

Before running a model, you need to convert this type of categorical text data into numerical data that the model can understand.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

**Format submission : un fichier + 5 sec + un oiseau --> TRUE or FALSE**

**PREPARE TEST DATA**

In [ ]:
# # Create empty csv file 
# header = "filename length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var label".split()

# file = open('datatest.csv', 'w', newline = '')
# with file:
#     writer = csv.writer(file)
#     writer.writerow(header)
    
# # Transform each .wav file into a .csv row:
# for line in test_data.index:
#     sound_name = path+'test_soundscapes/'+test_data['file_id'][line]
#     y, sr = librosa.load(sound_name, mono = True, duration = 30)
#     chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
#     rmse = librosa.feature.rms(y = y)
#     spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
#     spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
#     rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
#     zcr = librosa.feature.zero_crossing_rate(y)
#     mfcc = librosa.feature.mfcc(y = y, sr = sr)
#     to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

#     for e in mfcc:
#         to_append += f' {np.mean(e)}'
        
#     to_append += str(" ") + str(test_data['bird'][line])
#     file = open('datatest.csv', 'a', newline = '')
    
#     with file:
#         writer = csv.writer(file)
#         writer.writerow(to_append.split())    
    
# df = pd.read_csv('datatest.csv')

In [ ]:
# Create empty csv file 
header = "filename length chroma_stft_mean chroma_stft_var rms_mean rms_var spectral_centroid_mean spectral_centroid_var spectral_bandwidth_mean spectral_bandwidth_var rolloff_mean rolloff_var zero_crossing_rate_mean zero_crossing_rate_var harmony_mean harmony_var perceptr_mean perceptr_var tempo mfcc1_mean mfcc1_var mfcc2_mean mfcc2_var mfcc3_mean mfcc3_var mfcc4_mean mfcc4_var".split()

file = open('datatest.csv', 'w', newline = '')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

files = sorted(os.listdir(path+'test_soundscapes/'))

with open('../input/birdclef-2022/scored_birds.json') as fp:
    SCORED_BIRDS = json.load(fp)

In [ ]:
import math

In [ ]:
def create_df_test_from_path():
    files = sorted(os.listdir(path+'test_soundscapes/'))
    data = []
    for f in files:
        wv, sr = librosa.load(path+'test_soundscapes/' + f)
        n_chunks = math.ceil(len(wv) / sr / 5)
        filename = f
        row_prefix = f[:-4]
#         bird = SCORED_BIRDS[0]
        for chunk in range(1, n_chunks + 1):
            for bird in SCORED_BIRDS:
                row_id = f"{f[:-4]}_{bird}_{chunk*5}"
            
                ending_second = chunk*5
                data.append((filename, row_prefix, ending_second, bird))
            
    return  pd.DataFrame(data, columns=['filename', 'row_prefix', 'ending_second', 'birds'])
        
test_df = create_df_test_from_path()


In [ ]:
test_df

In [ ]:
# Transform each .wav file into a .csv row:
# for line in test_df.index:
#     sound_name = path+'test_soundscapes/'+test_df['filename'][line]
for f in files:
    sound_name = path+'test_soundscapes/'+f
    print(sound_name)
    y, sr = librosa.load(sound_name, mono = True, duration = 60)
    chroma_stft = librosa.feature.chroma_stft(y = y, sr = sr)
    rmse = librosa.feature.rms(y = y)
    spec_cent = librosa.feature.spectral_centroid(y = y, sr = sr)
    spec_bw = librosa.feature.spectral_bandwidth(y = y, sr = sr)
    rolloff = librosa.feature.spectral_rolloff(y = y, sr = sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y = y, sr = sr)
    to_append = f'{f} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

    for e in mfcc:
        to_append += f' {np.mean(e)}'
        
#     to_append += str(" ") + str(test_data['bird'][line])
    file = open('datatest.csv', 'a', newline = '')
    
    with file:
        writer = csv.writer(file)
        writer.writerow(to_append.split())    
    
df = pd.read_csv('datatest.csv')

ICI UNE PREDICTION PAR FICHIER ET PAS PAR 5SEC --> IL FAUT INTEGRER LA POSSIBILIT2 QU4IL N4Y AI AUCUN OISEAU

In [ ]:
from sklearn.preprocessing import StandardScaler

fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:, 2:27], dtype=float))
X

In [ ]:
y_pred  = modelv1.predict(X)
classes = np.argmax(y_pred, axis = 1)

In [ ]:
# transform classes number into classes name
result = converter.inverse_transform(classes)
print(result)

In [ ]:
# insert preditcion is the dataframe
df.insert(0, 'result', result)

In [ ]:
test_df = pd.merge(test_df, df[['filename','result']], how='left', left_on = 'filename', right_on = 'filename')

In [ ]:
test_df

In [ ]:
from tqdm.auto import tqdm

In [ ]:
import json

submission = []
for i, row in tqdm(test_df.iterrows(), total=len(test_df)):
    preds = True if test_df['birds'][i] == test_df['result'][i] else False
    submission.append({
        "row_id": f"{row['row_prefix']}_{row['birds']}_{row['ending_second']}",
        "target": preds#[bird] > 1. / len(model.labels),
    })

df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")